In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style
from webdriver_manager.chrome import ChromeDriverManager

print(plt.style.available)
%matplotlib inline
driver = webdriver.Chrome()

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


# 一、二手房网站

In [3]:
# 爬取二手房数据
esf_data = []
url = 'https://tj.esf.fang.com/house-a052/'
driver.get(url)
sleep(5)

In [4]:
i = 0
Num_Pages = 20
while i < Num_Pages:
    try:
        # 等待页面完全加载
        sleep(5)
        houses = driver.find_elements(By.CSS_SELECTOR, ".shop_list > dl")
        page_data = []  # 临时存储当前页数据
        
        for house in houses:
            try:
                tel_shop_elements = house.find_elements(By.CLASS_NAME, "tel_shop")
                price_elements = house.find_elements(By.CSS_SELECTOR, ".price_right .red b")
                unit_price_elements = house.find_elements(By.CSS_SELECTOR, ".price_right span:not(.red)")
                
                if tel_shop_elements and price_elements and unit_price_elements:
                    tel_text = tel_shop_elements[0].text
                    price_text = price_elements[0].text
                    unit_price_text = unit_price_elements[0].text
                    
                    import re
                    area_match = re.search(r'(\d+\.?\d*)㎡', tel_text)
                    
                    if area_match:
                        area_text = area_match.group(1)
                        
                        page_data.append({
                            '面积(㎡)': float(area_text),
                            '总价(万)': float(price_text),
                            '单价(元/㎡)': int(unit_price_text.replace("元/㎡", "").replace(",", ""))
                        })
            except Exception as e:
                continue
        
        # 将当前页数据添加到总数据中
        esf_data.extend(page_data)
        print(f"第{i+1}页成功提取 {len(page_data)} 条数据")
        
        # 翻页
        next_page = driver.find_element(By.CSS_SELECTOR, "p.last a")
        next_page.click()
        sleep(3)
        i += 1
        
    except Exception as e:
        print(f"错误: {e}")
        break

print(f"总共收集到 {len(esf_data)} 条数据")

第1页成功提取 60 条数据
第2页成功提取 60 条数据
第3页成功提取 60 条数据
第4页成功提取 60 条数据
第5页成功提取 60 条数据
第6页成功提取 60 条数据
第7页成功提取 60 条数据
第8页成功提取 60 条数据
第9页成功提取 60 条数据
第10页成功提取 60 条数据
第11页成功提取 60 条数据
第12页成功提取 60 条数据
第13页成功提取 60 条数据
第14页成功提取 60 条数据
第15页成功提取 60 条数据
第16页成功提取 60 条数据
第17页成功提取 60 条数据
第18页成功提取 60 条数据
第19页成功提取 60 条数据
第20页成功提取 60 条数据
总共收集到 1200 条数据


In [5]:
df_esf = pd.DataFrame(esf_data)
df_esf.to_excel("Tianjin_Wuqing_esf_data.xlsx", index=False)

# 二、租房网站

In [7]:
# 爬取租房数据
zu_data = []
url = 'https://tj.zu.fang.com/house-a052/'
driver.get(url)
sleep(5)

In [8]:
j = 0
Zu_Pages = 20
while j < Zu_Pages:
    try:
        # 等待页面完全加载
        sleep(3) 
        houses = driver.find_elements(By.CSS_SELECTOR, "dd.info.rel")
        page_data = []  # 临时存储当前页数据
        
        for house in houses:
            try:
                area_elements = house.find_elements(By.CSS_SELECTOR, ".font15.mt12.bold")
                price_elements = house.find_elements(By.CSS_SELECTOR, ".price")
                
                if area_elements and price_elements:
                    area_text = area_elements[0].text
                    price_text = price_elements[0].text
                    
                    import re
                    area_match = re.search(r'(\d+)㎡', area_text)
                    
                    if area_match:
                        area_value = int(area_match.group(1))
                        price_value = int(price_text)
                        
                        page_data.append({
                            '面积(㎡)': area_value,
                            '租金(元/月)': price_value
                        })
            except Exception as e:
                continue
        
        # 将当前页数据添加到总数据中
        zu_data.extend(page_data)
        print(f"第{j+1}页成功提取 {len(page_data)} 条数据")
        
        # 翻页
        next_page = driver.find_element(By.XPATH, "//a[contains(text(),'下一页')]")
        next_page.click()
        sleep(3)
        j += 1
        
    except Exception as e:
        print(f"错误: {e}")
        break

print(f"租房数据总共收集到 {len(zu_data)} 条数据")

第1页成功提取 60 条数据
第2页成功提取 60 条数据
第3页成功提取 60 条数据
第4页成功提取 60 条数据
第5页成功提取 60 条数据
第6页成功提取 60 条数据
第7页成功提取 60 条数据
第8页成功提取 60 条数据
第9页成功提取 60 条数据
第10页成功提取 60 条数据
第11页成功提取 60 条数据
第12页成功提取 60 条数据
第13页成功提取 60 条数据
第14页成功提取 60 条数据
第15页成功提取 60 条数据
第16页成功提取 60 条数据
第17页成功提取 60 条数据
第18页成功提取 60 条数据
第19页成功提取 60 条数据
第20页成功提取 60 条数据
租房数据总共收集到 1200 条数据


In [9]:
df_zu = pd.DataFrame(zu_data)
df_zu.to_excel("Tianjin_Wuqing_zu_data.xlsx", index=False)